In [13]:
import pandas as pd
import numpy as np
import random
import optuna
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

import pickle

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
train_df = pd.read_csv('./output/中間データ/train_preprocessed.csv')
train_df.head()

,datetime,y,week,soldout,name,kcal,remarks,payday,weather,precipitation,...,y_rolling_mean_7,y_rolling_max_7,y_rolling_min_7,y_rolling_mean_14,y_rolling_max_14,y_rolling_min_14,kcal_missing_flag,weather_group,event_キャリアアップ支援セミナー,event_ママの会
0,2013-11-18,90.0,-1.449157,0,厚切りイカフライ,-352.889193,0,0.0,-1.231166,-0.174563,...,NaN,NaN,NaN,NaN,NaN,NaN,1,-0.89977,False,False
1,2013-11-19,101.0,-0.734930,1,手作りヒレカツ,-352.889193,0,0.0,-1.231166,-0.174563,...,NaN,NaN,NaN,NaN,NaN,NaN,1,-0.89977,False,False
2,2013-11-20,118.0,-0.020702,0,白身魚唐揚げ野菜あん,-352.889193,0,0.0,-1.231166,-0.174563,...,NaN,NaN,NaN,NaN,NaN,NaN,1,-0.89977,False,False
3,2013-11-21,120.0,0.693525,1,若鶏ピリ辛焼,-352.889193,0,0.0,-1.231166,-0.174563,...,NaN,NaN,NaN,NaN,NaN,NaN,1,-0.89977,False,False
4,2013-11-22,130.0,1.407752,1,ビッグメンチカツ,-352.889193,0,0.0,-1.231166,-0.174563,...,NaN,NaN,NaN,NaN,NaN,NaN,1,-0.89977,False,False


In [3]:
X = train_df.drop(columns=['y', 'name', 'datetime'])
X = X.astype(np.float32)
y = train_df['y']

In [4]:
# 時系列CVの1 fold目を使ってOptuna用の学習・検証セットを作る
tscv = TimeSeriesSplit(n_splits=5)
train_idx, valid_idx = list(tscv.split(X))[0]
X_train, X_valid = X.iloc[train_idx].values, X.iloc[valid_idx].values
y_train, y_valid = y[train_idx], y[valid_idx]

In [5]:
# 目的関数
def objective(trial):
    n_layers = trial.suggest_int("n_layers", 2, 4)
    units = trial.suggest_int("units", 64, 256)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

    model = Sequential()
    model.add(Dense(units, activation='relu', input_shape=(X_train.shape[1],)))
    for _ in range(n_layers - 1):
        model.add(Dropout(dropout_rate))
        model.add(Dense(units, activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

    # 欠損行除外
    train_mask = ~np.isnan(X_train).any(axis=1)
    val_mask = ~np.isnan(X_valid).any(axis=1)

    X_train_clean = X_train[train_mask]
    y_train_clean = y_train[train_mask]
    X_valid_clean = X_valid[val_mask]
    y_valid_clean = y_valid[val_mask]

    model.fit(
        X_train_clean, y_train_clean,
        validation_data=(X_valid_clean, y_valid_clean),
        epochs=100,
        batch_size=batch_size,
        verbose=0,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
    )

    y_pred = model.predict(X_valid_clean).flatten()
    return mean_squared_error(y_valid_clean, y_pred)

In [9]:
# Optunaによるチューニング
sampler = optuna.samplers.TPESampler(seed=1234)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=30)

[I 2025-05-02 17:02:18,018] A new study created in memory with name: no-name-df7e745b-5758-41d3-85a8-9b035b3e8cc5
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


[I 2025-05-02 17:02:21,786] Trial 0 finished with value: 85.03894917829551 and parameters: {'n_layers': 2, 'units': 184, 'dropout_rate': 0.27509109560284584, 'learning_rate': 0.0037214926657368383, 'batch_size': 32}. Best is trial 0 with value: 85.03894917829551.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


[I 2025-05-02 17:02:25,916] Trial 1 finished with value: 181.42179873115245 and parameters: {'n_layers': 4, 'units': 248, 'dropout_rate': 0.4503730538968379, 'learning_rate': 0.000519558602432567, 'batch_size': 128}. Best is trial 0 with value: 85.03894917829551.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

[I 2025-05-02 17:03:52,025] Trial 27 finished with value: 42.716082475488825 and parameters: {'n_layers': 4, 'units': 203, 'dropout_rate': 0.14520140993764413, 'learning_rate': 0.0030520097465026276, 'batch_size': 128}. Best is trial 27 with value: 42.716082475488825.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


[I 2025-05-02 17:03:54,795] Trial 28 finished with value: 87.78381087072536 and parameters: {'n_layers': 4, 'units': 243, 'dropout_rate': 0.147343107301181, 'learning_rate': 0.003277961085104865, 'batch_size': 32}. Best is trial 27 with value: 42.716082475488825.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


[I 2025-05-02 17:03:57,252] Trial 29 finished with value: 160.81026316509474 and parameters: {'n_layers': 4, 'units': 198, 'dropout_rate': 0.10126223171275986, 'learning_rate': 0.005486328753064492, 'batch_size': 64}. Best is trial 27 with value: 42.716082475488825.


In [15]:
# ベストパラメータの取得（Optuna後）
best_params = study.best_params

SEED = 1234
def set_seed(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_seed()

# 最終モデルの構築
def build_final_model(input_dim, params):
    model = Sequential()
    model.add(Dense(params['units'], activation='relu', input_dim=input_dim))
    for _ in range(params['n_layers'] - 1):
        model.add(Dropout(params['dropout_rate']))
        model.add(Dense(params['units'], activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=params['learning_rate']), loss='mse')
    return model

# 再学習（全データ）
X_train = train_df.drop(columns=['y', 'name', 'datetime'])
y_train = train_df['y']

# NaNがある行を除外（X, y 両方に適用）
mask = ~X_train.isna().any(axis=1)
X_train_clean = X_train[mask].astype(np.float32)
y_train_clean = y_train[mask]

final_model = build_final_model(X_train_clean.shape[1], best_params)
final_model.fit(
    X_train_clean, y_train_clean,
    epochs=100,
    batch_size=best_params['batch_size'],
    verbose=0,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]
)

# トレーニングRMSEを計算
y_train_pred = final_model.predict(X_train_clean).flatten()
final_rmse = np.sqrt(mean_squared_error(y_train_clean, y_train_pred))
print(f"最終NNモデルのトレーニングRMSE: {final_rmse:.4f}")

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/callbacks/early_stopping.py:153: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  current = self.get_monitor_value(logs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
最終NNモデルのトレーニングRMSE: 10.2993


In [16]:
# モデルを保存
with open('./output/モデル/nn_model.pkl', 'wb') as f:
    pickle.dump(final_model, f)